# Julia High-Performance

## BenchmarkTools

In [1]:
using BenchmarkTools

## 01. Loop

**Q. Implement below procedure as julia function**
$$ \sum_{k=1}^n \sqrt{k} $$

In [2]:
function sumsqrtn(n)
    s = 0
    for k = 1:n
        s += sqrt(k)
    end
    return s
end

sumsqrtn (generic function with 1 method)

In [3]:
# Measure performance
@btime sumsqrtn(1000_000)

  1.909 ms (0 allocations: 0 bytes)


6.666671664588418e8

In [4]:
@code_warntype sumsqrtn(1000_000)

Variables
  #self#::Core.Compiler.Const(sumsqrtn, false)
  n::Int64
  s::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64

Body::Union{Float64, Int64}
1 ─       (s = 0)
│   %2  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (k = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Union{Float64, Int64}
│   %11 = Main.sqrt(k)::Float64
│         (s = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [5]:
function sumsqrtn_fixed(n)
    s = 0.0
    for k = 1:n
        s += sqrt(k)
    end
    return s
end

sumsqrtn_fixed (generic function with 1 method)

In [6]:
@btime sumsqrtn_fixed(1000_000)

  1.399 ms (0 allocations: 0 bytes)


6.666671664588418e8

In [7]:
@code_warntype sumsqrtn_fixed(1000_000)

Variables
  #self#::Core.Compiler.Const(sumsqrtn_fixed, false)
  n::Int64
  s::Float64
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64

Body::Float64
1 ─       (s = 0.0)
│   %2  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (k = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = Main.sqrt(k)::Float64
│         (s = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [8]:
function simd_sum(x)
    s = 0
    @simd for v in x
        s += v
    end
    return s
end

simd_sum (generic function with 1 method)

In [9]:
x = sqrt.(Float32[1:1000_000...]);

In [10]:
@btime simd_sum(x)

  1.041 ms (1 allocation: 16 bytes)


6.669686f8

In [11]:
function simd_sum_fixed(x)
    s = zero(eltype(x))
    @simd for v in x
        s += v
    end
    return s
end

simd_sum_fixed (generic function with 1 method)

In [12]:
@btime simd_sum_fixed(x)

  66.213 μs (1 allocation: 16 bytes)


6.6666726f8

In [13]:
y = sqrt.(Float64[1:1000_000...]);

In [14]:
@btime simd_sum_fixed(y)

  167.876 μs (1 allocation: 16 bytes)


6.666671664588225e8

## 02. Matrix Initialization

**Q. Initialize $1000\times1000$ matrix**

In [15]:
@btime zeros(1000, 1000);

  505.504 μs (2 allocations: 7.63 MiB)


In [16]:
@btime Array{Float64}(undef, 1000, 1000)

  14.392 μs (2 allocations: 7.63 MiB)


1000×1000 Array{Float64,2}:
 4.6432e-310   0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 6.91065e-310  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.6432e-310   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.6432e-310   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0           0.0  0.0  0.0  0.0  0.0     0.0  

## 03. Matrix Iteration

Julia uses Column-wise storage as default

In [17]:
function col_iter(x)
    s = zero(eltype(x))
    for j in 1:size(x, 2)
        for i in 1:size(x, 1)
            s += x[i, j]^2
            x[i, j] = s
        end
    end
end

col_iter (generic function with 1 method)

In [18]:
function row_iter(x)
    s = zero(eltype(x))
    for i in 1:size(x, 1)
        for j in 1:size(x, 2)
            s += x[i, j]^2
            x[i, j] = s
        end
    end
end

row_iter (generic function with 1 method)

In [19]:
a = rand(1000, 1000);
b = a';

In [20]:
@btime col_iter(a);

  1.024 ms (0 allocations: 0 bytes)


In [21]:
@btime row_iter(a);

  1.414 ms (0 allocations: 0 bytes)


In [22]:
@btime col_iter(b);

  1.367 ms (0 allocations: 0 bytes)


In [23]:
@btime row_iter(b);

  1.009 ms (0 allocations: 0 bytes)


## 04. Const

In [24]:
p = 2

2

In [25]:
function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [26]:
function pow_array_generic(x::Vector{T}) where T <: Number
    s = zero(eltype(x))
    for y in x
        s += y^p
    end
    return s
end

pow_array_generic (generic function with 1 method)

In [27]:
function pow_array_simd(x::Vector{T}) where T <: Number
    s = zero(eltype(x))
    @simd for y in x
        s += y^p
    end
    return s
end

pow_array_simd (generic function with 1 method)

In [28]:
t = rand(100_000);

In [29]:
@btime pow_array(t);

  4.157 ms (300000 allocations: 4.58 MiB)


In [30]:
@btime pow_array_generic(t);

  4.247 ms (300000 allocations: 4.58 MiB)


In [31]:
@btime pow_array_simd(t);

  4.036 ms (300000 allocations: 4.58 MiB)


In [32]:
const p2 = 2

2

In [33]:
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p2
    end
    return s
end

pow_array2 (generic function with 1 method)

In [34]:
function pow_array2_generic(x::Vector{T}) where T <: Number
    s = zero(eltype(x))
    for y in x
        s += y^p2
    end
    return s
end

pow_array2_generic (generic function with 1 method)

In [35]:
function pow_array2_simd(x::Vector{T}) where T <: Number
    s = zero(eltype(x))
    @simd for y in x
        s += y^p2
    end
    return s
end

pow_array2_simd (generic function with 1 method)

In [36]:
@btime pow_array2(t);

  95.521 μs (0 allocations: 0 bytes)


In [37]:
@btime pow_array2_generic(t);

  100.291 μs (1 allocation: 16 bytes)


In [38]:
@btime pow_array2_simd(t);

  12.287 μs (1 allocation: 16 bytes)


## 06. Struct

In [39]:
abstract type AbstractPoints end

In [40]:
struct Point2D <: AbstractPoints
    x
    y
end

In [41]:
function sumsqr_points(a::Vector{T}) where T <: AbstractPoints
    s = 0.0
    for t in a
        s += t.x^2 + t.y^2
    end
    return s
end

sumsqr_points (generic function with 1 method)

In [42]:
p_array = [Point2D(rand(), rand()) for i in 1:1000_000];

In [43]:
@btime sumsqr_points(p_array)

  80.141 ms (4000000 allocations: 61.04 MiB)


666150.6955756759

In [44]:
@code_warntype sumsqr_points(p_array)

Variables
  #self#::Core.Compiler.Const(sumsqr_points, false)
  a::Array{Point2D,1}
  s::Any
  @_4::Union{Nothing, Tuple{Point2D,Int64}}
  t::Point2D

Body::Any
1 ─       (s = 0.0)
│   %2  = a::Array{Point2D,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Point2D,Int64}::Tuple{Point2D,Int64}
│         (t = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = Base.getproperty(t, :x)::Any
│   %12 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %13 = (%12)()::Core.Compiler.Const(Val{2}(), false)
│   %14 = Base.literal_pow(Main.:^, %11, %13)::Any
│   %15 = Base.getproperty(t, :y)::Any
│   %16 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %17 = (%16)()::Core.Compiler.Const(Val{2}(), false)
│   %18 = Base.literal_pow(Main.:^, %15, %17)::Any
│   %19 = (%14 + %18)::Any
│         (s = %10 + %19)
│         (@_4

In [45]:
struct ConcretePoint2D <: AbstractPoints
    x::Float64
    y::Float64
end

In [46]:
cp_array = [ConcretePoint2D(rand(), rand()) for i in 1:1000_000];

In [48]:
@btime sumsqr_points(cp_array)

  1.270 ms (1 allocation: 16 bytes)


665551.54868266

In [49]:
struct Point2DWithAbstract <: AbstractPoints
    x::AbstractFloat
    y::AbstractFloat
end

In [51]:
struct ParametricPoint2D{T <: AbstractFloat} <: AbstractPoints
    x::T
    y::T
end

In [52]:
pa_array = [Point2DWithAbstract(rand(), rand()) for i in 1:1000_000];
pp_array = [ParametricPoint2D(rand(), rand()) for i in 1:1000_000];

In [54]:
@btime sumsqr_points(pa_array);

  92.965 ms (4000000 allocations: 61.04 MiB)


In [55]:
@btime sumsqr_points(pp_array);

  1.315 ms (1 allocation: 16 bytes)


In [58]:
function sumsqr_points_simd(a::Vector{T}) where T <: AbstractPoints
    s = zero(eltype(a[1].x))
    @simd for t in a
        s += t.x^2 + t.y^2
    end
    return s
end

sumsqr_points_simd (generic function with 1 method)

In [59]:
@btime sumsqr_points_simd(pp_array);

  726.654 μs (1 allocation: 16 bytes)


In [60]:
g(x) = map(t -> t + 1, x)

g (generic function with 1 method)

In [61]:
test = rand(1000_000);

In [63]:
@code_warntype g(test);

Variables
  #self#::Core.Compiler.Const(g, false)
  x::Array{Float64,1}
  #34::var"#34#35"

Body::Array{Float64,1}
1 ─      (#34 = %new(Main.:(var"#34#35")))
│   %2 = #34::Core.Compiler.Const(var"#34#35"(), false)
│   %3 = Main.map(%2, x)::Array{Float64,1}
└──      return %3


In [65]:
@btime g(test);

  667.839 μs (2 allocations: 7.63 MiB)


In [72]:
function g_fixed(x)
    s = Array{eltype(x)}(undef, length(x))
    for (i, t) in enumerate(x)
        s[i] = t + 1
    end
    return s
end

g_fixed (generic function with 1 method)

In [73]:
@btime g_fixed(test);

  660.105 μs (2 allocations: 7.63 MiB)


In [74]:
g_sum(x) = sum(map(t -> sqrt(t), x));

In [82]:
@btime g_sum(test)

  1.815 ms (3 allocations: 7.63 MiB)


666771.7171670175

In [79]:
function g_sum_simd(x)
    s = zero(eltype(x))
    @simd for t in x
        s += t
    end
    return s
end

g_sum_simd (generic function with 1 method)

In [83]:
@btime g_sum_simd(sqrt.(test))

  1.845 ms (5 allocations: 7.63 MiB)


666771.7171670191

In [84]:
@btime sum(sqrt.(test))

  1.880 ms (5 allocations: 7.63 MiB)


666771.7171670175

In [86]:
test2 = sqrt.(test);

In [87]:
@btime g_sum_simd(test2)

  142.074 μs (1 allocation: 16 bytes)


666771.7171670191

In [88]:
@btime sum(test2)

  146.833 μs (1 allocation: 16 bytes)


666771.7171670175